In [1]:
import soundfile
from espnet2.train.trainer import Trainer
from espnet2.tasks.asr import ASRTask
import argparse
from pathlib import Path
import yaml
from espnet2.train.distributed_utils import DistributedOption
from espnet2.utils.build_dataclass import build_dataclass
from espnet2.torch_utils.device_funcs import to_device

ModuleNotFoundError: No module named 'soundfile'

In [2]:
ckpt_path = '/data/anakuzne/espnet/egs2/librispeech_100/asr1/exp/asr_codec_frozen_full_lr2e-3_conv_input_large_batch_warmup15k_from_pretrained_num_enc_layers_1/71epoch.pth'
config_path = '/data/anakuzne/espnet/egs2/librispeech_100/asr1/exp/asr_codec_frozen_full_lr2e-3_conv_input_large_batch_warmup15k_from_pretrained_num_enc_layers_1/config.yaml'

In [3]:
task = ASRTask
model_latest, asr_train_args = task.build_model_from_file(
            config_path, ckpt_path, "cuda:7"
        )


In [4]:
with Path(config_path).open("r", encoding="utf-8") as f:
            args = yaml.safe_load(f)
args = argparse.Namespace(**args)


In [5]:
distributed_option = build_dataclass(DistributedOption, args)
# Setting distributed_option.dist_rank, etc.
distributed_option.init_options()

In [6]:
train_iter_factory = task.build_iter_factory(
                    args=args,
                    distributed_option=distributed_option,
                    mode="train",
                )

valid_iter_factory = task.build_iter_factory(
                    args=args,
                    distributed_option=distributed_option,
                    mode="valid",
                )

In [8]:
data_loader = train_iter_factory.build_iter(1000)
data_loader_valid = valid_iter_factory.build_iter(1000)

In [13]:
model_latest.eval()
avg_train_len = 0
for iiter, (utt_id, batch) in enumerate(data_loader):
    #batch = to_device(batch, "cuda:7")
    #print(batch["speech"].shape)
    #print(iiter, utt_id, batch)
    #retval = model_latest(**batch)
    #print("Train Loss ", retval[0])
    avg_train_len += batch['speech'].shape[1]
    
print(avg_train_len/len(data_loader))

683.8096816976127


In [11]:
print(model_latest)

ESPnetASRModel(
  (specaug): SpecAug(
    (time_warp): TimeWarp(window=5, mode=bicubic)
    (freq_mask): MaskAlongAxis(mask_width_range=[0, 27], num_mask=2, axis=freq)
    (time_mask): MaskAlongAxisVariableMaxWidth(mask_width_ratio_range=[0.0, 0.05], num_mask=5, axis=time)
  )
  (normalize): UtteranceMVN(norm_means=True, norm_vars=False)
  (encoder): ConformerEncoder(
    (embed): Conv2dSubsampling(
      (conv): Sequential(
        (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
        (3): ReLU()
      )
      (out): Sequential(
        (0): Linear(in_features=65280, out_features=256, bias=True)
        (1): RelPositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (encoders): MultiSequential(
      (0): EncoderLayer(
        (self_attn): RelPositionMultiHeadedAttention(
          (linear_q): Linear(in_features=256, out_features=256, bias=True)
    

In [9]:
avg_len = 0
model_latest.eval()
for iiter, (utt_id, batch) in enumerate(data_loader_valid):
    batch = to_device(batch, "cuda:7")
    #print(iiter, utt_id, batch)
    retval = model_latest(**batch)
    print("Valid Loss True", retval[0].data,  "Len ", batch["speech"].shape[1])
        
    if iiter == 100:
        break
    #avg_len += batch["speech"].shape[1]
#print(avg_len//len(data_loader_valid))

Valid Loss True tensor([614.0464], device='cuda:7') Len  1758
Valid Loss True tensor([637.0003], device='cuda:7') Len  1608
Valid Loss True tensor([572.8401], device='cuda:7') Len  1507
Valid Loss True tensor([460.9959], device='cuda:7') Len  1420
Valid Loss True tensor([501.3669], device='cuda:7') Len  1349
Valid Loss True tensor([482.5706], device='cuda:7') Len  1240
Valid Loss True tensor([450.1767], device='cuda:7') Len  1193
Valid Loss True tensor([459.9008], device='cuda:7') Len  1141
Valid Loss True tensor([376.3450], device='cuda:7') Len  1070
Valid Loss True tensor([391.4791], device='cuda:7') Len  1037
Valid Loss True tensor([368.1139], device='cuda:7') Len  1011
Valid Loss True tensor([366.3072], device='cuda:7') Len  988
Valid Loss True tensor([354.7173], device='cuda:7') Len  956
Valid Loss True tensor([325.1338], device='cuda:7') Len  925
Valid Loss True tensor([314.3872], device='cuda:7') Len  909
Valid Loss True tensor([325.4771], device='cuda:7') Len  890
Valid Loss Tr

In [ ]:
print(model_latest)

In [16]:
model_ave_best.eval()
batch = to_device(batch, "cuda:6")
retval = model_ave_best(**batch)
print(retval[0])

tensor([186.4736], device='cuda:6', grad_fn=<UnsqueezeBackward0>)


In [23]:
model_ave_best.eval()
batch = to_device(batch, "cuda:6")
retval = model_ave_best(**batch)
print(retval[0])

tensor([178.8518], device='cuda:6', grad_fn=<UnsqueezeBackward0>)


In [20]:
model_ave_best10.eval()
batch = to_device(batch, "cuda:6")
retval = model_ave_best10(**batch)
print(retval[0])

tensor([186.4736], device='cuda:6', grad_fn=<UnsqueezeBackward0>)


In [21]:
import os
import shutil
import random

In [99]:
d = "/data/common/LibriSpeech/train-clean-100-reduced"

all_paths = []
for root, dirs, files in os.walk(d):
    for f in files:
        if ".txt" in f:
        #if "flac" in f:
            p = f"{root}/{f}"
            all_paths.append(p)

In [100]:
all_paths[:5]

['/data/common/LibriSpeech/train-clean-100-reduced/103/1240/103-1240.trans.txt',
 '/data/common/LibriSpeech/train-clean-100-reduced/103/1241/103-1241.trans.txt',
 '/data/common/LibriSpeech/train-clean-100-reduced/1034/121119/1034-121119.trans.txt',
 '/data/common/LibriSpeech/train-clean-100-reduced/1040/133433/1040-133433.trans.txt',
 '/data/common/LibriSpeech/train-clean-100-reduced/1069/133699/1069-133699.trans.txt']

In [92]:
random.shuffle(all_paths)

In [93]:
train_set = all_paths[:-5000]
dev_set = all_paths[-5000:]

In [94]:
len(train_set)

23539

In [95]:
dest = 'dev-from-train'
ls_path = '/data/common/LibriSpeech'

In [72]:
dev_set[:4]

['/data/common/LibriSpeech/train-clean-100-reduced/6476/57446/6476-57446-0062.wav',
 '/data/common/LibriSpeech/train-clean-100-reduced/226/122538/226-122538-0024.wav',
 '/data/common/LibriSpeech/train-clean-100-reduced/7226/86964/7226-86964-0054.wav',
 '/data/common/LibriSpeech/train-clean-100-reduced/3607/135982/3607-135982-0010.wav']

In [102]:
for f in all_paths:
    s = f.split('/')[-3:]
    spk, chapter, fname = s
    dest_path = f"{ls_path}/{dest}/{spk}/{chapter}/"
    #if not os.path.exists(dest_path):
    #    os.makedirs(dest_path)
    #shutil.move(f, dest_path)
    #print(dest_path)
    source_path = f"{d}/{spk}/{chapter}/{spk}-{chapter}.trans.txt"
    #print(source_path, dest_path)
    try:
        shutil.copy(source_path, dest_path)
    except FileNotFoundError:
        print("File is not in dev set")
        continue

File is not in dev set
File is not in dev set
File is not in dev set
File is not in dev set


In [2]:
import dac
from audiotools import AudioSignal

In [3]:
model_path = dac.utils.download(model_type="16khz")
model = dac.DAC.load(model_path)

model.to('cuda:5')

/data/anakuzne/espnet/tools/enc_asr/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


DAC(
  (encoder): Encoder(
    (block): Sequential(
      (0): Conv1d(1, 64, kernel_size=(7,), stride=(1,), padding=(3,))
      (1): EncoderBlock(
        (block): Sequential(
          (0): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(3,))
              (2): Snake1d()
              (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
            )
          )
          (1): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
              (2): Snake1d()
              (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
            )
          )
          (2): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(27,), dilation=(9,))
              (2): Snake1d()
              

In [4]:
# Load audio signal file
signal = AudioSignal('/data/anakuzne/LibriSpeech/train-clean-100/201/122255/201-122255-0000.wav')

# Encode audio signal as one long file
# (may run out of GPU memory on long files)
signal.to(model.device)

x = model.preprocess(signal.audio_data, signal.sample_rate)
z, codes, latents, _, _ = model.encode(x)

In [5]:
encoder = model.encoder

In [7]:
quantizer = model.quantizer

In [8]:
print(quantizer)

ResidualVectorQuantize(
  (quantizers): ModuleList(
    (0-11): 12 x VectorQuantize(
      (in_proj): Conv1d(1024, 8, kernel_size=(1,), stride=(1,))
      (out_proj): Conv1d(8, 1024, kernel_size=(1,), stride=(1,))
      (codebook): Embedding(1024, 8)
    )
  )
)


In [12]:
z = encoder(signal.audio_data)

In [13]:
z.shape

torch.Size([1, 1024, 246])

In [16]:
signal.audio_data

tensor([[[ 0.0004,  0.0003, -0.0003,  ...,  0.0024, -0.0024,  0.0013]]],
       device='cuda:5')